<a href="https://colab.research.google.com/github/kelvinr99/xml2docs/blob/main/XML_Reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf spark-3.2.4-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop3.2"

!ls

import findspark
findspark.init()


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [46.6 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [49.8 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,419 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadc

In [ ]:
#!wget -q https://repo1.maven.org/maven2/com/databricks/spark-xml_2.12/0.17.0/spark-xml_2.12-0.17.0.jar


In [6]:

from pyspark.sql import SparkSession
import xml.etree.ElementTree as ET
spark = SparkSession.builder \
        .master("local").appName("XML Reader") \
        .getOrCreate()

def get_documents(xml_doc):
  books = []
  row = xml_doc.asDict()
  xml_root = ET.fromstring(row['value'])
  for book in xml_root.findall('book'):
   books.append({'document':ET.tostring(book, encoding='utf8')})
  return books

file_rdd = spark.read.text("/content/sample_data/xmldata*", wholetext=True).rdd
results = file_rdd.flatMap(lambda x: get_documents(x))
results.take(5)

# use insertInto having stripped out the partitions, have to default the load
# possibly to the datetime
# want to do is strip out the id from the metadata

AnalysisException: ignored

In [90]:
spark

In [ ]:
xmldata = spark.read.format("xml") \
            .option("rootTag", "catalog") \
            .option("rowTag", "book") \
            .load("/content/sample_data/xmldata")

In [ ]:
xmldata.show()

+-----+--------------------+--------------------+--------+--------+-----+------------+--------------------+
|  _id|              author|         description|document|   genre|price|publish_date|               title|
+-----+--------------------+--------------------+--------+--------+-----+------------+--------------------+
|bk101|Gambardella, Matthew|An in-depth look ...|     {1}|Computer|44.95|  2000-10-01|XML Developer's G...|
|bk102|          Ralls, Kim|A former architec...|     {1}| Fantasy| 5.95|  2000-12-16|       Midnight Rain|
+-----+--------------------+--------------------+--------+--------+-----+------------+--------------------+



In [64]:
xmldata.collect()

[Row(_id='bk101', author='Gambardella, Matthew', description='An in-depth look at creating applications \n      with XML.', document=Row(id=1), genre='Computer', price=44.95, publish_date=datetime.date(2000, 10, 1), title="XML Developer's Guide"),
 Row(_id='bk102', author='Ralls, Kim', description='A former architect battles corporate zombies, \n      an evil sorceress, and her own childhood to become queen \n      of the world.', document=Row(id=1), genre='Fantasy', price=5.95, publish_date=datetime.date(2000, 12, 16), title='Midnight Rain')]

In [86]:
file_rdd = spark.read.text("/content/sample_data/xmldata*", wholetext=True).rdd

In [87]:
file_rdd.take(5)

[Row(value='<?xml version="1.0"?>\n<catalog>\n   <book id="bk101">\n      <document><id>1</id></document>\n      <author>Gambardella, Matthew</author>\n      <title>XML Developer\'s Guide</title>\n      <genre>Computer</genre>\n      <price>44.95</price>\n      <publish_date>2000-10-01</publish_date>\n      <description>An in-depth look at creating applications \n      with XML.</description>\n   </book>\n   <book id="bk102">\n      <document><id>1</id></document>\n      <author>Ralls, Kim</author>\n      <title>Midnight Rain</title>\n      <genre>Fantasy</genre>\n      <price>5.95</price>\n      <publish_date>2000-12-16</publish_date>\n      <description>A former architect battles corporate zombies, \n      an evil sorceress, and her own childhood to become queen \n      of the world.</description>\n   </book>\n</catalog>   '),
 Row(value='<?xml version="1.0"?>\n<catalog>\n   <book id="bk101">\n      <document><id>1</id></document>\n      <author>Gambardella, Matthew</author>\n      <

In [50]:
file = spark.read.text("/content/sample_data/xmldata*", wholetext=True)

In [51]:
file.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                       

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
root = ET.fromstring(file_rdd.take(1)[0][0])

In [ ]:
for i, child in enumerate(root[0]):
  print(i)
  for value in child:
    for val in value:
      print(val.tag, " :", value.find(value.tag).text)

0
1
2
3
4
5
6


In [88]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
@udf
def get_documents(xml_doc):
  thedict = xml_doc.asDict()
  results = []
  return results

results = file_rdd.flatMap(get_documents)
results.take(5)

Py4JJavaError: ignored

In [162]:
meta = spark.read.text("/content/sample_data/metadata", wholetext=True)


In [158]:
meta.show()

+--------------------+
|               value|
+--------------------+
|<?xml version="1....|
+--------------------+



In [167]:

row = meta.collect()[0]
xml_root = ET.fromstring(row['value'])
load = xml_root.find('load').text
graph = xml_root.find('graph').text
print(load)
print(graph)

123456
abc
